# Recommendation Tool Creation
# Integrating with Gradio interface
---
This Colab Notebook reads in the csv file that contains all movies that

In [ ]:
!pip install pandas
!pip install gradio

In [3]:
import gradio as gr
import pandas as pd

# Read the recommendation_ready.csv file into the dataframe for use
movie_predictions_df = pd.read_csv("recommendation_ready.csv")

In [ ]:
movie_predictions_df.head()

In [7]:
movie_predictions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6123 entries, 0 to 6122
Data columns (total 44 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            6123 non-null   int64  
 1   genre_ids             6123 non-null   object 
 2   movie_id              6123 non-null   int64  
 3   popularity            6123 non-null   float64
 4   release_date          6123 non-null   object 
 5   title                 6123 non-null   object 
 6   clean_reviews         6122 non-null   object 
 7   type_Action           6123 non-null   int64  
 8   type_Adventure        6123 non-null   int64  
 9   type_Animation        6123 non-null   int64  
 10  type_Comedy           6123 non-null   int64  
 11  type_Crime            6123 non-null   int64  
 12  type_Documentary      6123 non-null   int64  
 13  type_Drama            6123 non-null   int64  
 14  type_Family           6123 non-null   int64  
 15  type_Fantasy         

In [10]:
# with open('movie_output.txt', 'r') as f:
#     lines = f.readlines()

# print(lines)

['Emotions: Neutral\n', 'Genres: Comedy, Adventure, Action\n']


In [12]:
import pandas as pd

def recommend_movies(file):
    # Read the file and parse the emotion and genres
    with open(file.name, 'r') as f:
        lines = f.readlines()

    # Extract emotion and genres from the file content
    emotion = lines[0].split(': ')[1].strip().lower() # Convert emotion to lowercase
    genres = [genre.strip().lower() for genre in lines[1].split(': ')[1].strip().split(', ')] # Convert each genre to lowercase

    # Filter movies based on the predicted emotion
    emotion_movies = movie_predictions_df[movie_predictions_df['label'].str.lower() == emotion]

    base_url = "https://image.tmdb.org/t/p/"
    file_size = "w300"


    # If no movies match the emotion, return a message
    if emotion_movies.empty:
        return [{"message": "No movies match your emotion preference."}]

    # Further filter movies based on the selected genres
    genre_movies = emotion_movies.copy()
    for genre in genres:
        # Adjust the genre name to match the DataFrame column format
        genre_column = f"type_{genre.capitalize().replace(' ', '_')}"
        genre_movies = genre_movies[genre_movies[genre_column] == 1]

    # If no movies match the genres, return a message
    if genre_movies.empty:
        return [{"message": "No movies match your genre preferences."}]

    # Rank movies based on the number of criteria they match and emotion score
    ranked_movies = []
    for index, row in genre_movies.iterrows():
        # Add the emotion score to the match count
        match_count = row[emotion]

        # Construct the poster URL
        poster_url = f"{base_url}{file_size}{row['poster_path']}"
        # Add the movie to the ranked list
        ranked_movies.append((row['movie_id'], row['title'], poster_url, match_count))

    # Sort the ranked list by match count (descending)
    ranked_movies.sort(key=lambda x: x[3], reverse=True)

    # Select the top 3 movies
    top_movies = ranked_movies[:3]

    # Format the recommendations for display
    recommendations = [{
        "id": movie[0],
        "title": movie[1],
        "poster_url": movie[2],
        "overview":
        "emotion": emotion.capitalize(),
        "genres": ", ".join(genres)
    } for movie in top_movies]

    return recommendations

# Example usage
# Assuming the file 'movie_output.txt' is in the same directory as this script
with open('movie_output.txt', 'r') as file:
    recommendations = recommend_movies(file)

# Print the recommendations
for rec in recommendations:
    print(f"Movie ID: {rec['id']}, Title: {rec['title']}, Poster URL: {rec['poster_url']}, Emotion: {rec['emotion']}, Genres: {rec['genres']}")


Movie ID: 9326, Title: Romancing the Stone, Poster URL: https://image.tmdb.org/t/p/w300/qfIlVOYZH5RiZ0ndFKUpr5Zn2Bx.jpg, Emotion: Neutral, Genres: comedy, adventure, action
Movie ID: 354857, Title: Regular Show: The Movie, Poster URL: https://image.tmdb.org/t/p/w300/p44RoW4naoE1nlSGOqqT0J2pc09.jpg, Emotion: Neutral, Genres: comedy, adventure, action
Movie ID: 38319, Title: Your Highness, Poster URL: https://image.tmdb.org/t/p/w300/bq3gU60TkgBjLUbxSSZVhDhlhmH.jpg, Emotion: Neutral, Genres: comedy, adventure, action


In [4]:
# movie_output.txt

In [8]:
# line 1 emotions:[]
# line 2 genres:[]